In [1]:
import tkinter as tk
from tkinter import ttk, messagebox, filedialog
import numpy as np
from joblib import load
import warnings

predict_root = tk.Tk()
predict_root.title("Optimization Prediction Interface")
predict_root.geometry("1400x1000")
predict_root.overrideredirect(True)

close_button = ttk.Button(predict_root, text="X", command=predict_root.destroy, width=3)
close_button.place(relx=1.0, rely=1.0, x=-30, y=-30, anchor="se")

warnings.filterwarnings('ignore', category=UserWarning)

models = {
    'Objective1-CS': load('CS-CatBoost-model.joblib'),
    'Objective2-FS': load('FS-CatBoost-model.joblib'),
    'Objective3-SD': load('SD-CatBoost-model.joblib'),
    'Objective4-SK': load('SK-XGBoost-model.joblib')
}

feature_mapping = {
    'Objective1-CS': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,18], 
    'Objective2-FS': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,16,18],
    'Objective3-SD': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14],
    'Objective4-SK': [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,17,18,19,20]
}

def aim_function(**kwargs):
    individual = np.array([kwargs[var] for var in input_labels])
    
    if not _check_constraints(individual):
        raise ValueError("Input does not meet constraints.")
    
    try:
        obj1 = _predict_model('Objective1-CS', individual)
        obj2 = _predict_model('Objective2-FS', individual)
        obj3 = _predict_model('Objective3-SD', individual)
        obj4 = _predict_model('Objective4-SK', individual)
        
        variables = _extract_variables(individual)
        obj5 = _calculate_cost(variables)
        obj6 = _calculate_GWP(variables)
        obj7 = _calculate_EE(variables)
        obj8 = _calculate_ODP(variables)
        obj9 = _calculate_AP(variables)
        obj10 = _calculate_EP(variables)
        
        return [obj1, obj2, obj3, obj4, obj5, obj6, obj7, obj8, obj9, obj10]
        
    except Exception as e:
        raise Exception(f"Prediction failed: {str(e)}")

def _check_constraints(individual):
    sum_val = (
        individual[0] + individual[1] + individual[2] + individual[3] +
        individual[4] + individual[5] + individual[6] +
        individual[7] + individual[8] + individual[10] +
        individual[11] + individual[12] + individual[13]
    )
    return 2000 <= sum_val <= 3500

def _predict_model(model_name, individual):
    features = individual[feature_mapping[model_name]].reshape(1, -1)
    return models[model_name].predict(features)[0]

def _extract_variables(individual):
    return {
        'PC': individual[0],
        'FA': individual[1],
        'SSP': individual[2],
        'SL': individual[3],
        'LP': individual[4],
        'ME': individual[5],
        'RHA': individual[6],
        'SF': individual[7],
        'FI': individual[8],
        'LE': individual[9],
        'SA': individual[10],
        'CA': individual[11],
        'WA': individual[12],
        'SP': individual[13],
        'WB': individual[14]
    }

def _calculate_cost(var): 
    return (0.110*var['PC'] + 0.025*var['FA'] + 0.015*var['SSP'] + 
            0.100*var['SL'] + 0.120*var['LP'] + 0.500*var['ME'] + 
            0.015*var['RHA'] + 0.800*var['SF'] + 4.760*var['FI'] + 
            0.025*var['SA'] + 0.045*var['CA'] + 0.001*var['WA'] + 
            1.200*var['SP'])

def _calculate_GWP(var):
    return (0.927*var['PC'] + 8.88e-3*var['FA'] + 0.027*var['SSP'] + 
            0.052*var['SL'] + 0.019*var['LP'] + 0.400*var['ME'] + 
            0.145*var['RHA'] + 0.004*var['SF'] + 1.490*var['FI'] + 
            9.87e-3*var['SA'] + 0.029*var['CA'] + 1.33e-4*var['WA'] + 
            0.720*var['SP'])

def _calculate_EE(var):
    return (5.800*var['PC'] + 0.830*var['FA'] + 1.100*var['SSP'] + 
            1.590*var['SL'] + 0.760*var['LP'] + 3.480*var['ME'] + 
            0.103*var['RHA'] + 0.036*var['SF'] + 20.56*var['FI'] + 
            0.110*var['SA'] + 0.055*var['CA'] + 0.006*var['WA'] + 
            18.3*var['SP'])

def _calculate_ODP(var):
    return (9.47e-8*var['PC'] + 5.58e-9*var['FA'] + 6.49e-9*var['SSP'] + 
            1.82e-8*var['SL'] + 5.72e-9*var['LP'] + 1.52e-9*var['ME'] + 
            1.85e-10*var['RHA'] + 1.21e-11*var['SF'] + 1.39e-7*var['FI'] + 
            1.71e-11*var['SA'] + 4.08e-10*var['CA'] + 5.93e-12*var['WA'] + 
            3.29e-8*var['SP'])

def _calculate_AP(var):
    return (2.55e-3*var['PC'] + 5.53e-5*var['FA'] + 1.61e-4*var['SSP'] + 
            1.03e-3*var['SL'] + 1.24e-4*var['LP'] + 3.24e-5*var['ME'] + 
            8.10e-5*var['RHA'] + 1.90e-6*var['SF'] + 1.40e-2*var['FI'] + 
            4.58e-5*var['SA'] + 1.63e-4*var['CA'] + 9.70e-7*var['WA'] + 
            1.19e-2*var['SP'])

def _calculate_EP(var):
    return (3.52e-4*var['PC'] + 7.28e-6*var['FA'] + 1.61e-4*var['SSP'] + 
            1.48e-4*var['SL'] + 9.22e-6*var['LP'] + 4.89e-5*var['ME'] + 
            3.20e-5*var['RHA'] + 1.29e-7*var['SF'] + 1.00e-3*var['FI'] + 
            1.08e-5*var['SA'] + 3.74e-5*var['CA'] + 4.99e-8*var['WA'] + 
            5.97e-3*var['SP'])

def update_predict():
    try:
        input_values = {}
        for var_name, var in input_vars.items():
            input_values[var_name] = float(var.get())
        
        if not (0 <= input_values['PC (kg/m³)'] <= 1933 and 
                0 <= input_values['FA (kg/m³)'] <= 704 and 
                0 <= input_values['SSP (kg/m³)'] <= 200 and 
                0 <= input_values['SL (kg/m³)'] <= 957.8 and 
                0 <= input_values['LP (kg/m³)'] <= 1058.2 and 
                0 <= input_values['ME (kg/m³)'] <= 308 and 
                0 <= input_values['RHA (kg/m³)'] <= 276 and 
                0 <= input_values['SF (kg/m³)'] <= 583 and 
                0 <= input_values['FI (kg/m³)'] <= 470 and 
                0 <= input_values['LE (mm)'] <= 30 and 
                0 <= input_values['SA (kg/m³)'] <= 1460.3 and 
                0 <= input_values['CA (kg/m³)'] <= 1200 and 
                0 <= input_values['WA (kg/m³)'] <= 387 and 
                0 <= input_values['SP (kg/m³)'] <= 53 and 
                0.125 <= input_values['WB (--)'] <= 0.285 and 
                10.86 <= input_values['CV (cm³)'] <= 3375 and 
                256 <= input_values['FV (cm³)'] <= 12375 and 
                160 <= input_values['SKSL (mm)'] <= 515 and 
                1 <= input_values['Age (days)'] <= 365 and 
                20 <= input_values['Tem  (℃)'] <= 25 and 
                50 <= input_values['Hum (--)'] <= 60):
            raise ValueError("Input error, please check value ranges.")
        
        sum_val = (input_values['PC (kg/m³)'] + input_values['FA (kg/m³)'] + input_values['SSP (kg/m³)'] + input_values['SL (kg/m³)'] +
                   input_values['LP (kg/m³)'] + input_values['ME (kg/m³)'] + input_values['RHA (kg/m³)'] + input_values['SF (kg/m³)'] +
                   input_values['FI (kg/m³)'] + input_values['SA (kg/m³)'] + input_values['CA (kg/m³)'] + input_values['WA (kg/m³)'])
        if not (2000 <= sum_val <= 3500):
            raise ValueError("Input error, total must be between 2000 and 3500.")
        
        results = aim_function(**input_values)
        
        for i, (result_var, result) in enumerate(zip(output_vars, results)):
            if i == 7:
                result_var.delete(0, tk.END)
                result_var.insert(0, f"{result:.2e}")
            else:
                result_var.delete(0, tk.END)
                result_var.insert(0, f"{result:.2f}")
    
    except ValueError as e:
        messagebox.showerror("Error", str(e))
    except Exception as e:
        messagebox.showerror("Error", str(e))

def clear_entries():
    for var in input_vars.values():
        var.set("")
    for entry in output_vars:
        entry.delete(0, tk.END)

def save_entries():
    file_path = filedialog.asksaveasfilename(defaultextension=".txt", filetypes=[("Text Files", "*.txt"), ("All Files", "*.*")])
    if file_path:
        with open(file_path, "w") as file:
            input_data = ""
            for var_name, var in input_vars.items():
                input_data += f"{var_name}: {var.get()}\n"
            
            output_data = ""
            for i, entry in enumerate(output_vars):
                output_data += f"Objective {i+1}: {entry.get()}\n"
            
            file.write(input_data + "\n" + output_data)
            messagebox.showinfo("Save Successful", "Data saved successfully.")

def return_main_menu():
    predict_root.destroy()

title_label = tk.Label(predict_root, text="Optimization Prediction Interface of UHPC", font=('Times New Roman', 24, "bold"))
title_label.grid(row=0, column=0, columnspan=3, sticky="ew", padx=10, pady=10)

style = ttk.Style(predict_root)
style.configure("TFrame", borderwidth=5, relief="groove")

input_outer_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
input_outer_frame.grid(row=1, column=0, sticky="nsew", padx=5)

input_title_label = ttk.Label(input_outer_frame, text="Input panel", font=('Times New Roman', 20, "bold"))
input_title_label.pack(fill="x", padx=310, pady=5)

input_frame = ttk.Frame(input_outer_frame, padding="6 6 6 6")
input_frame.pack(fill="both", expand=True, padx=5, pady=5)

input_vars = {}
input_labels = ['PC (kg/m³)','FA (kg/m³)','SSP (kg/m³)','SL (kg/m³)','LP (kg/m³)','ME (kg/m³)','RHA (kg/m³)','SF (kg/m³)','FI (kg/m³)','LE (mm)','SA (kg/m³)','CA (kg/m³)','WA (kg/m³)','SP (kg/m³)','WB (--)','CV (cm³)','FV (cm³)','SKSL (mm)','Age (days)','Tem  (℃)','Hum (--)']

for i, label in enumerate(input_labels):
    row = i // 3
    col = i % 3
    ttk.Label(input_frame, text=f"{label}", font=('Times New Roman', 14)).grid(row=row, column=col*2, padx=17, pady=25, sticky="ew")
    input_vars[label] = tk.StringVar()
    ttk.Entry(input_frame, textvariable=input_vars[label], width=10, font=('Times New Roman', 14)).grid(row=row, column=col*2+1, padx=10, pady=15, sticky="ew")

output_outer_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
output_outer_frame.grid(row=1, column=2, sticky="nsew", padx=5)

output_title_label = ttk.Label(output_outer_frame, text="Output panel", font=('Times New Roman', 20, "bold"))
output_title_label.pack(fill="x", padx=220, pady=5)

output_frame = ttk.Frame(output_outer_frame, padding="0 0 0 0")
output_frame.pack(fill="both", expand=True, padx=5, pady=5)

output_labels = ['CS (MPa)','FS (MPa)','SD (mm)','SK (με)','Cost (USD/m³)','GWP (kg CO₂/m³)','EE (MJ/m³)','ODP (kg CFC/m³)','AP (kg SO₂/m³)','EP (kg PO₄³⁻/m³)']
output_vars = []

for i, label in enumerate(output_labels):
    row = i // 2
    col = i % 2
    ttk.Label(output_frame, text=f"{label}", font=('Times New Roman', 14)).grid(row=row, column=col*2, padx=18, pady=42.5, sticky="ew")
    entry = ttk.Entry(output_frame, width=10, font=('Times New Roman', 14))
    entry.grid(row=row, column=col*2+1, padx=20, pady=25, sticky="ew")
    output_vars.append(entry)

control_panel_frame = ttk.Frame(predict_root, padding="0 0 0 0", style="TFrame")
control_panel_frame.grid(row=2, column=0, columnspan=5, sticky="ew", padx=5, pady=5)

control_panel_title_label = ttk.Label(control_panel_frame, text="Control panel", font=('Times New Roman', 20, "bold"))
control_panel_title_label.pack(fill="x", padx=600, pady=5)

control_panel_inner_frame = ttk.Frame(control_panel_frame, padding="0 0 0 0")
control_panel_inner_frame.pack(fill="x", padx=5, pady=5)

style = ttk.Style(predict_root)
style.configure("TFrame", borderwidth=5, relief="groove")
style.configure("TLabel", font=('Times New Roman', 14))
style.configure("TButton", 
                background='#0078D4',
                foreground='black',
                bordercolor='#0078D4',
                borderwidth=1,
                font=('Times New Roman', 18, "bold"))
style.map('TButton',
          background=[('active', '#005A9E'), ('pressed', '#004A8C')],
          foreground=[('active', 'white'), ('pressed', 'white')])

button_predict = ttk.Button(control_panel_inner_frame, text="Predict", command=update_predict, width=15, style='TButton')
button_predict.pack(side="left", padx=(120, 12), pady=20, anchor="n")

button_clear = ttk.Button(control_panel_inner_frame, text="Clear", command=clear_entries, width=15, style='TButton')
button_clear.pack(side="left", padx=(120, 12), pady=20, anchor="n")

button_save = ttk.Button(control_panel_inner_frame, text="Save", command=save_entries, width=15, style='TButton')
button_save.pack(side="left", padx=(120, 12), pady=20, anchor="n")

button_main_menu = ttk.Button(control_panel_inner_frame, text="Exit", command=return_main_menu, width=15, style='TButton')
button_main_menu.pack(side="left", padx=(120, 12), pady=20, anchor="n")

predict_root.columnconfigure(0, weight=4)
predict_root.columnconfigure(2, weight=1)

predict_root.mainloop()